In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torchtext
import sys
!{sys.executable} -m pip freeze| grep torch
from torchtext import data
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import re, os.path
import gensim
from gensim.models import KeyedVectors
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print('device={}'.format(device))

torch==0.4.1
torchtext==0.3.1
torchvision==0.2.1
device=cuda:0


# Transformar base CETEN para TSV

In [2]:
%%time
labels = ['N', 'PROP', 'ADJ','V', 'PERS', 'DET', 'SPEC', 'ADV', 'NUM', 'PRP', 'IN', 'KS', 'KC', 'EC']
text_tag_re = re.compile(r'^(.+?)\s.*?\s(' + '|'.join(labels) + ')\s')
CETEN_path = 'resource/CETENFolha-1.0_jan2014.cg'
train_path = 'resource/CETENFolha-1.0_jan2014.cg.tsv'
def build_sentence(sentence_lines):
    text = []
    labels = []
    for sentence_line in sentence_lines:
        for match in text_tag_re.finditer(sentence_line):
            text.append(match.groups()[0])
            labels.append(match.groups()[1])
    return text, labels

def sentence_lines_iterator(filename):
    with open(filename, 'r') as f:
        sentence_lines = []
        for line in f:
            if line.startswith('</s>'):
                text, labels = build_sentence(sentence_lines)
                if len(text) > 0:
                    yield text, labels
                sentence_lines = []
            else:
                sentence_lines.append(line)            
        f.close()

def persist_tsv(filename_from, filename_to):
    if os.path.isfile(filename_to):
        return
    with open(filename_to, 'w+') as fw:
        fw.write('text\tlabel\n')
        for text, labels in sentence_lines_iterator(filename_from):
            text_col = ' '.join(text)
            label_col = ' '.join(labels)
            fw.write(text_col + '\t' + label_col + '\n')
        fw.close()
persist_tsv(CETEN_path, train_path)

CPU times: user 537 µs, sys: 0 ns, total: 537 µs
Wall time: 621 µs


# Carregar e dividir em treino e teste

In [3]:
%%time
txt_field = torchtext.data.Field(sequential=True, 
                       include_lengths=True, 
                       use_vocab=True)
label_field = torchtext.data.Field(sequential=True, 
                         use_vocab=False)
train_val_fields = [
    ('text', txt_field),
    ('label', label_field)
]
trainds = torchtext.data.TabularDataset(path=train_path, 
                                            format='tsv',
                                            fields=train_val_fields, 
                                            skip_header=True)
# TODO Encontrar um jeito rápido de fazer shuffle e depois split 

CPU times: user 1.34 ms, sys: 606 µs, total: 1.95 ms
Wall time: 1.9 ms


In [4]:
%%time
txt_field.build_vocab(trainds, max_size=500000)
label_field.build_vocab(trainds)

CPU times: user 4.15 ms, sys: 79 µs, total: 4.23 ms
Wall time: 4.15 ms


In [5]:
print('Tamanho do vocabulário:', len(txt_field.vocab))
print('número de classes:', len(label_field.vocab))
for i, word in enumerate(txt_field.vocab.itos[:10]):
    print(word, i)

Tamanho do vocabulário: 778
número de classes: 14
<unk> 0
<pad> 1
de 2
o 3
a 4
em 5
que 6
e 7
os 8
para 9


In [6]:
for word in 'PT em o governo'.split(' '):
    print(word, txt_field.vocab.stoi[word])

PT 25
em 5
o 3
governo 21


In [7]:
train, valid = trainds.split(0.8)

In [8]:
print('número de dados de treinamento:', len(train))
print('número de dados de validação:', len(valid))
#print('número de dados de teste:', len(test))

número de dados de treinamento: 79
número de dados de validação: 20


In [10]:
%%time
wv_path = 'resource/CETENFolha-1.0_jan2014.cg.wv'
if not os.path.isfile(wv_path):
    sentences = [x for x in trainds.text]
    model = gensim.models.Word2Vec (sentences, size=150, window=10, workers=10, min_count=1)
    model.train(sentences,total_examples=len(sentences),epochs=4)
    model.wv.save(wv_path)
    del model
    del sentences
wv = KeyedVectors.load(wv_path, mmap='r')
label_to_ix = {label: i for i, label in enumerate(labels)}

CPU times: user 6.74 ms, sys: 0 ns, total: 6.74 ms
Wall time: 6.16 ms


In [13]:
# ignorar
print(wv.index2word[0])
#print(wv.vectors[0])
#print(wv['de'] == wv.vectors[0])
weights = torch.FloatTensor(wv.vectors)
em = nn.Embedding.from_pretrained(weights)
lookup_tensor = torch.tensor([0], dtype=torch.long)
#print(em(['de'])[0])
#print(wv.vectors[0])
# txt_field.build_vocab(trainds, valds, max_size=len(wv.index2word), vectors=wv.vectors)

#embedding = nn.Embedding.from_pretrained(weights)
#label_field.build_vocab(trainds)
#print(wv['A'])
#my_iterator = [x for x in trainds.text]
#print(my_iterator[0])
#txt_field.build_vocab(trainds)
#vectors = torchtext.vocab.Vectors(wv_path)
#txt_field.vocab.set_vectors(vectors.stoi, vectors.vectors, vectors.dim)
#embedding = nn.Embedding.from_pretrained(torch.FloatTensor(txt_field.vocab.vectors))


#print(len(wv.index2word))
#txt_field.build_vocab(trainds, max_size=len(wv.index2word), vectors=wv)
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([0], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)


de
tensor([[-1.0430,  1.5876,  1.1685,  0.6252, -0.3075]], grad_fn=<EmbeddingBackward>)
tensor([[-1.0430,  1.5876,  1.1685,  0.6252, -0.3075]], grad_fn=<EmbeddingBackward>)


NameError: name 'batch_size' is not defined

In [17]:
batch_size = 10
train_iter = torchtext.data.BucketIterator(train, batch_size=batch_size, repeat=True)
valid_iter = torchtext.data.BucketIterator(valid, batch_size=batch_size, repeat=False)
#test_iter = torchtext.data.BucketIterator(valds, batch_size=len(valds), repeat=False)

In [18]:
#for batch in train_iter.__iter__():
#bi = data.BucketIterator(trainds, batch_size=10, repeat=False)
#print(dir(bi))
bi_iter = iter(valid_iter)
#batch = next(iter(train))
for batch in bi_iter:
    print("batch.text.shape = {}".format(batch.text.shape))
    print("batch.text.type = {}\n".format(batch.text.type()))
    print("batch.label.shape = {}".format(batch.label.shape))
    print("batch.label.type = {}\n".format(batch.label.type()))
    break

ValueError: too many dimensions 'str'

In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
        
        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
        
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out

#input_dim = np.prod(train_dataset[0][0].size()) #(1, 28, 28)
#input_dim = train_dataset[0][0].size()[1]
input_dim = 150
hidden_dim = batch_size
layer_dim = 1
output_dim = 16

In [22]:
def calculate_accuracy(iter, loss, elapsed_seconds):
    correct = 0
    total = 0    
    for images, labels in test_loader:
        images = images.view(-1, input_dim, input_dim).requires_grad_().to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += int((predicted.cpu() == labels.cpu()).sum())
    accuracy = 100 * correct / total
    iter_seconds = iter/elapsed_seconds
    remaining_seconds = round((n_iters - iter)/iter_seconds)
    perc_done = round(iter/n_iters * 100)
    print('iteration={} of {} ({}%), loss={}, accuracy={}%, elapsed_seconds={}, remaning_seconds={}'.format(iter, n_iters, perc_done, round(loss.item(), 2), accuracy, elapsed_seconds, remaining_seconds))

In [26]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model.to(device)

print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

num_epochs = 5 # qtd vezes que treinará sobre todo o dataset
n_iters = int(len(train) / batch_size * num_epochs)

iter = 0
start_time = time.time()
for epoch in range(num_epochs):
    loss = None
    for i, batch in enumerate(train):
        print(batch)
        images = batch.text.view(-1, input_dim, input_dim).requires_grad_().to(device)
        labels = batch.label.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter += 1
        if iter % 500 == 0:
            calculate_accuracy(iter, loss, round(time.time() - start_time))


LSTMModel(
  (lstm): LSTM(300, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=16, bias=True)
)
6
torch.Size([400, 300])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([16, 100])
torch.Size([16])


AttributeError: 'list' object has no attribute 'view'